# Dependencies

In [8]:
# import libraries
import requests
import time
from datetime import datetime
import json
import sys
import pandas as pd
pd.set_option('display.max_columns', 500)  #allows display of all columns in dfs

import numpy as np
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [9]:
# mysql login info
sys.path.insert(0, '../../Key')
from mysql_secret import dbuser, dbpass, dbhost, dbname
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')

# Scrape IPOs
Retrieve IPO stock information from Nasdaq

In [ ]:
current_year_month = datetime.today().strftime('%Y-%m')
current_year_month

In [ ]:
# scrape nasdaq, Ex: https://api.nasdaq.com/api/ipo/calendar?date=2020-08
# note, had to create headers due to time out, solution found here: https://stackoverflow.com/questions/46862719/pythons-requests-library-timing-out-but-getting-the-response-from-the-browser
url = f'https://api.nasdaq.com/api/ipo/calendar?date={current_year_month}'
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15","Accept-Language": "en-gb","Accept-Encoding":"br, gzip, deflate","Accept":"test/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Referer":"http://www.google.com/"}

response = requests.get(url, headers=headers)
data = response.text
data = json.loads(data)
print(url)

In [ ]:
# create list of scraped dataframes to concatenate
scraped_ipo_dfs = []

In [ ]:
# gets priced IPOs for each record on nasdaq (values) = [(expression) for (value) in (collection)]
priced_ipos = data["data"]["priced"]["rows"]

# if there are priced IPOs create dataframe
if priced_ipos:
    
    symbols = [priced_ipos[x]["proposedTickerSymbol"] for x in range(len(priced_ipos))]
    company = [priced_ipos[x]["companyName"] for x in range(len(priced_ipos))]
    exchange = [priced_ipos[x]["proposedExchange"] for x in range(len(priced_ipos))]
    proposed_share_price = [priced_ipos[x]["proposedSharePrice"] for x in range(len(priced_ipos))]
    shares_offered = [priced_ipos[x]["sharesOffered"].replace(",", '') for x in range(len(priced_ipos))]
    priced_date = [priced_ipos[x]["pricedDate"] for x in range(len(priced_ipos))]
    dollar_val_shares = [priced_ipos[x]["dollarValueOfSharesOffered"].replace(",", '').replace("$",'') for x in range(len(priced_ipos))]

    # dataframe with stock info
    nasdaq_priced_df = pd.DataFrame({"symbol" : symbols,
                                       "company" : company,
                                       "exchange" : exchange, 
                                       "proposed_share_price" : proposed_share_price,
                                       "shares_offered" : shares_offered,
                                       "priced_date" : priced_date,
                                       "dollar_val_shares" : dollar_val_shares,
                                       "deal_status" : "priced"
                                       })
    scraped_ipo_dfs.append(nasdaq_priced_df)
    print(f"{len(priced_ipos)} priced IPOs")
else:
    print("no priced IPOs")

In [ ]:
# gets lists for each record on nasdaq (values) = [(expression) for (value) in (collection)]

upcoming_ipos = data["data"]["upcoming"]["upcomingTable"]["rows"]

if upcoming_ipos:
    symbols = [upcoming_ipos[x]["proposedTickerSymbol"] for x in range(len(upcoming_ipos))]
    company = [upcoming_ipos[x]["companyName"] for x in range(len(upcoming_ipos))]
    exchange = [upcoming_ipos[x]["proposedExchange"] for x in range(len(upcoming_ipos))]
    proposed_share_price = [upcoming_ipos[x]["proposedSharePrice"] for x in range(len(upcoming_ipos))]
    shares_offered = [upcoming_ipos[x]["sharesOffered"].replace(",", '') for x in range(len(upcoming_ipos))]
    priced_date = [upcoming_ipos[x]["expectedPriceDate"] for x in range(len(upcoming_ipos))]
    dollar_val_shares = [upcoming_ipos[x]["dollarValueOfSharesOffered"].replace(",", '').replace("$",'') for x in range(len(upcoming_ipos))]

    # dataframe with stock info
    nasdaq_upcoming_df = pd.DataFrame({"symbol" : symbols,
                                       "company" : company,
                                       "exchange" : exchange, 
                                       "proposed_share_price" : proposed_share_price,
                                       "shares_offered" : shares_offered,
                                       "priced_date" : priced_date,
                                       "dollar_val_shares" : dollar_val_shares,
                                       "deal_status" : "expected"
                                       })
    scraped_ipo_dfs.append(nasdaq_upcoming_df)
    print(f"{len(upcoming_ipos)} upcoming IPOs")
else: 
    print("no upcoming IPOs")

In [ ]:
# combine IPO dataframes
ipo_df = pd.concat(scraped_ipo_dfs, ignore_index=True, sort=False)

# change column datatypes
ipo_df[['shares_offered', 'dollar_val_shares']] = ipo_df[['shares_offered', 'dollar_val_shares']].apply(pd.to_numeric)
ipo_df['priced_date'] = pd.to_datetime(ipo_df['priced_date'], format="%m/%d/%Y")
ipo_df = ipo_df.sort_values(by='deal_status', ascending=False).reset_index(drop=True)
ipo_df = ipo_df.drop_duplicates(subset="symbol", keep="first")
print(ipo_df.dtypes)
ipo_df = ipo_df.dropna()

# update symbols with apostrophe.U issue (Ex: "NBA'U" as a symbol, they come in this format sometimes in Nasdaq and 
# in order to pull correctly from yahoo need the apostrophe.U removed
ipo_df["symbol"] = ipo_df["symbol"].str.replace("'U","")

# New IPOs to Database

In [ ]:
# Bring in ipo table
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')
connection = engine.connect()
sql_ipo_df = pd.read_sql("SELECT * FROM stocks", connection)
sql_ipo_df.head()

In [ ]:
new_ipos_df = ipo_df[~ipo_df["symbol"].isin(sql_ipo_df["symbol"])]
new_ipos_df.head()
new_ipos_df

In [ ]:
# load data
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')
new_ipos_df.to_sql('stocks', con=engine, if_exists='append', index=False)

# Update IPOs in Database
Any IPO that is deal status "expected" may have new data, so records should be updated to reflect most recent pull

In [ ]:
# query IPOs with expected status in SQL database
sql_expected = sql_ipo_df.query("deal_status == 'expected'")
sql_expected

In [ ]:
# expected stocks in SQL database that are in new IPO pull from nasdaq 
stocks_to_update = ipo_df[ipo_df["symbol"].isin(sql_expected["symbol"])]
stocks_to_update

In [ ]:
# create engine
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Print all of the classes mapped to the Base
Base.classes.keys()

In [ ]:
# Assign the ipo class to a variable called `IPO`
Stocks = Base.classes.stocks

# Create a session
session = Session(engine)

In [ ]:
# Use the session to query ipo table and display the first 5 symbols
for r in session.query(Stocks.symbol, Stocks.company).limit(5).all():
    print(r)

In [ ]:
for row in stocks_to_update.itertuples():

    # Get the record we want to change
    stock = session.query(Stocks).filter(Stocks.symbol==row.symbol).first()

    # Change the record
    stock.proposed_share_price = stocks_to_update.loc[stocks_to_update["symbol"] == row.symbol, 'proposed_share_price'].iloc[0]
    stock.shares_offered = stocks_to_update.loc[stocks_to_update["symbol"] == row.symbol, 'shares_offered'].iloc[0]
    stock.priced_date = stocks_to_update.loc[stocks_to_update["symbol"] == row.symbol, 'priced_date'].iloc[0]
    stock.dollar_val_shares = stocks_to_update.loc[stocks_to_update["symbol"] == row.symbol, 'dollar_val_shares'].iloc[0]
    stock.deal_status = stocks_to_update.loc[stocks_to_update["symbol"] == row.symbol, 'deal_status'].iloc[0]

# Update the database
session.commit()

# Stock Performance
To find these URLs go to a stock tickers page on Yahoo finance. When clicking on a tab open inspector > network > XHR > headers.  URL will display. 

General URL:  
https://query1.finance.yahoo.com/v8/finance/chart/AAPL?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&events=div%7Csplit&includeAdjustedClose=true&interval=1d&range=2y&corsDomain=finance.yahoo.com

URL with Unix Date Selection (period1=xxxxxxx&period2=xxxxxxxxxx):
https://query2.finance.yahoo.com/v8/finance/chart/AAPL?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1546300800&period2=1608076800&events=div%7Csplit&corsDomain=finance.yahoo.com

In [10]:
# get unix time range for web request
start_unixtime = 1514903400  #Jan 2, 2018

current_date = datetime.now()
print(current_date)

current_time = current_date.time()

# if its past 5p CST and markets are closed get unix to pull todays info, else pull market data from yesterday
if current_time.hour > 16:
    earliest_date_time = current_date.replace(hour=17, minute=0, second=0, microsecond=0)
else:
    earliest_date_time = current_date.replace(hour=0, minute=0, second=0, microsecond=0)
print(earliest_date_time)

2021-02-28 15:16:15.276970
2021-02-28 00:00:00


In [11]:
# change earliest data time to unix timestamp
end_unixtime = time.mktime(earliest_date_time.timetuple())
end_unixtime = int(end_unixtime)
end_unixtime

1614492000

In [12]:
today = datetime.today().strftime("%Y-%m-%d")
print(today)

2021-02-28


In [14]:
# query stocks from SQL
connection = engine.connect()
ipo_stocks = pd.read_sql(f"SELECT symbol, deal_status FROM stocks", connection)
ipo_stocks["default_start_unixtime"] = start_unixtime
ipo_stocks["end_unixtime"] = end_unixtime
ipo_stocks

,symbol,deal_status,default_start_unixtime,end_unixtime
0,AAAU,priced,1514903400,1614492000
1,AAC,priced,1514903400,1614492000
2,AACQU,priced,1514903400,1614492000
3,ABCL,priced,1514903400,1614492000
4,ABCM,priced,1514903400,1614492000
...,...,...,...,...
1208,ZNTEU,priced,1514903400,1614492000
1209,ZNTL,priced,1514903400,1614492000
1210,ZS,priced,1514903400,1614492000
1211,ZUO,priced,1514903400,1614492000


In [15]:
stock_perf_start_unix = pd.read_sql("SELECT symbol, max(unix_time) AS max_unix_captured  FROM performance  GROUP BY symbol""", connection)
ipo_stocks = ipo_stocks.merge(stock_perf_start_unix, on="symbol", how="outer")
ipo_stocks["max_unix_captured"] = ipo_stocks["max_unix_captured"].fillna(0).astype('int64')
ipo_stocks["max_unix_captured"] = ipo_stocks["max_unix_captured"] + 86400  #add a day to latest date captured
ipo_stocks

,symbol,deal_status,default_start_unixtime,end_unixtime,max_unix_captured
0,AAAU,priced,1514903400,1614492000,1614459600
1,AAC,priced,1514903400,1614492000,86400
2,AACQU,priced,1514903400,1614492000,1614459602
3,ABCL,priced,1514903400,1614492000,1614459602
4,ABCM,priced,1514903400,1614492000,1614459603
...,...,...,...,...,...
1208,ZNTEU,priced,1514903400,1614492000,1614459584
1209,ZNTL,priced,1514903400,1614492000,1614459603
1210,ZS,priced,1514903400,1614492000,1614459604
1211,ZUO,priced,1514903400,1614492000,1614459602


In [16]:
ipo_stocks["start_unixtime"] = ipo_stocks[["default_start_unixtime", "max_unix_captured"]].max(axis=1).astype('int64') 
ipo_stocks

,symbol,deal_status,default_start_unixtime,end_unixtime,max_unix_captured,start_unixtime
0,AAAU,priced,1514903400,1614492000,1614459600,1614459600
1,AAC,priced,1514903400,1614492000,86400,1514903400
2,AACQU,priced,1514903400,1614492000,1614459602,1614459602
3,ABCL,priced,1514903400,1614492000,1614459602,1614459602
4,ABCM,priced,1514903400,1614492000,1614459603,1614459603
...,...,...,...,...,...,...
1208,ZNTEU,priced,1514903400,1614492000,1614459584,1614459584
1209,ZNTL,priced,1514903400,1614492000,1614459603,1614459603
1210,ZS,priced,1514903400,1614492000,1614459604,1614459604
1211,ZUO,priced,1514903400,1614492000,1614459602,1614459602


In [23]:
# empty list of dfs
performance_df_list= []

# make requests for performance information for each row
for row in ipo_stocks.itertuples():

    url = f'https://query2.finance.yahoo.com/v8/finance/chart/{row.symbol}?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1={row.start_unixtime}&period2={row.end_unixtime}&events=div%7Csplit&corsDomain=finance.yahoo.com'
    r = requests.get(url)
    print(f"trying url: {url}")
    if r.ok:
        try: 
            data = r.json()

            # get data
            timestamp = data["chart"]["result"][0]["timestamp"]
            stk_open = data["chart"]["result"][0]["indicators"]["quote"][0]["open"]
            stk_close = data["chart"]["result"][0]["indicators"]["quote"][0]["close"]
            stk_high = data["chart"]["result"][0]["indicators"]["quote"][0]["high"]
            stk_low = data["chart"]["result"][0]["indicators"]["quote"][0]["low"]
            stk_vol = data["chart"]["result"][0]["indicators"]["quote"][0]["volume"]

            #transform into dataframe
            df = pd.DataFrame({"symbol" : row.symbol,
                               "unix_time" : timestamp,
                               "date" : [datetime.fromtimestamp(ts).strftime('%Y-%m-%d') for ts in timestamp],
                               "open" : stk_open, 
                               "close" : stk_close,
                               "high" : stk_high,
                               "low" : stk_low,
                               "volume" : stk_vol
                              })

            df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
            df['date_pulled'] = today

            print(f"{row.symbol} has results")
            performance_df_list.append(df)
        except KeyError:
            continue
    
    else:
        print(f"{row.symbol} NO RESULTS")

    time.sleep(.5)
    

trying url: https://query2.finance.yahoo.com/v8/finance/chart/AAAU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459600&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/AAC?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
AAC NO RESULTS
trying url: https://query2.finance.yahoo.com/v8/finance/chart/AACQU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459602&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/ABCL?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459602&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
tryin

trying url: https://query2.finance.yahoo.com/v8/finance/chart/AGCB?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459602&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/AGCUU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459603&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/AGFY?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459603&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/AGMH?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459602&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https:/

trying url: https://query2.finance.yahoo.com/v8/finance/chart/ANCN?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459603&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/ANDAU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614436200&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/ANGN?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459604&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/ANNX?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459602&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https:/

trying url: https://query2.finance.yahoo.com/v8/finance/chart/ASPCU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459599&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/ASPLU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
ASPLU NO RESULTS
trying url: https://query2.finance.yahoo.com/v8/finance/chart/ASZ?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/ATA?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459584&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
try

trying url: https://query2.finance.yahoo.com/v8/finance/chart/BCYPU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459549&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/BDRY?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459600&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/BDSX?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459603&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/BDTX?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459604&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https:/

...

trying url: https://query2.finance.yahoo.com/v8/finance/chart/TDACU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614458964&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/TEKKU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459387&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/TELA?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459603&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/TENB?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459603&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https:

trying url: https://query2.finance.yahoo.com/v8/finance/chart/TOTAU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1610116200&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/TPGYU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/TPTX?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459603&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/TREBU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
TREBU NO RESULTS


trying url: https://query2.finance.yahoo.com/v8/finance/chart/VCNX?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459604&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/VCTR?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459602&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/VCVCU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459603&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/VEL?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459602&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://

trying url: https://query2.finance.yahoo.com/v8/finance/chart/VYGGU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/WAFU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459604&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/WARRU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/WEI?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459721&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https:/

trying url: https://query2.finance.yahoo.com/v8/finance/chart/YSG?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459602&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/ZCMD?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459580&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/ZGYHU?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614449514&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://query2.finance.yahoo.com/v8/finance/chart/ZI?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1614459604&period2=1614492000&events=div%7Csplit&corsDomain=finance.yahoo.com
trying url: https://qu

In [24]:
# create one performance df from list of dfs
performance_df = pd.concat(performance_df_list)

In [25]:
# first cleaning step:  remove unix time not divisible by 100 as all desired data appears to have unix divisable by 100
# performance_df = performance_df.loc[test_df["unix_time"] % 100 == 0]

# second clearning step:  remove remaining DUPES
performance_df = performance_df.drop_duplicates(subset=['symbol', 'date'], keep='first').reset_index(drop=True)

performance_df.head(8)

,symbol,unix_time,date,open,close,high,low,volume,date_pulled
0,FSII,1614373202,2021-02-26,10.63,10.60,10.66,10.440,188421,2021-02-28
1,JYAC,1614373203,2021-02-26,10.67,10.52,10.67,10.380,30280,2021-02-28
2,KAIIU,1614373203,2021-02-26,10.06,10.06,10.14,10.040,428848,2021-02-28
3,KIIIU,1614373203,2021-02-26,10.19,10.13,10.19,10.060,501806,2021-02-28
4,LGVN,1614373203,2021-02-26,7.39,8.00,8.15,7.000,744223,2021-02-28
5,MACAU,1614373203,2021-02-26,10.41,10.32,10.45,10.275,161965,2021-02-28
6,NVSAU,1614373204,2021-02-26,10.67,10.53,10.86,10.490,908956,2021-02-28


In [22]:
# avoid adding a duplicate date
stock_perf_db = pd.read_sql("SELECT symbol, date FROM performance""", connection)
stock_perf_db["symbol_date"] = stock_perf_db["symbol"] + "_" + stock_perf_db["date"].astype(str)
stock_perf_db.head()

,symbol,date,symbol_date
0,AAAU,2018-08-15,AAAU_2018-08-15
1,AAAU,2018-08-16,AAAU_2018-08-16
2,AAAU,2018-08-17,AAAU_2018-08-17
3,AAAU,2018-08-20,AAAU_2018-08-20
4,AAAU,2018-08-21,AAAU_2018-08-21
...,...,...,...
302065,LIVKU,2021-02-26,LIVKU_2021-02-26
302066,MIT,2021-02-26,MIT_2021-02-26
302067,PHCF,2021-02-26,PHCF_2021-02-26
302068,WSG,2021-02-26,WSG_2021-02-26


In [28]:
# remove any symbols date duplicate combos
performance_df["symbol_date"] = performance_df["symbol"] + "_" + performance_df["date"].astype(str)
performance_df = performance_df[~performance_df["symbol_date"].isin(stock_perf_db[["symbol_date"]])]
performance_df.drop(columns="symbol_date", inplace=True)
performance_df

,symbol,unix_time,date,open,close,high,low,volume,date_pulled
0,FSII,1614373202,2021-02-26,10.63,10.60,10.66,10.440,188421,2021-02-28
1,JYAC,1614373203,2021-02-26,10.67,10.52,10.67,10.380,30280,2021-02-28
2,KAIIU,1614373203,2021-02-26,10.06,10.06,10.14,10.040,428848,2021-02-28
3,KIIIU,1614373203,2021-02-26,10.19,10.13,10.19,10.060,501806,2021-02-28
4,LGVN,1614373203,2021-02-26,7.39,8.00,8.15,7.000,744223,2021-02-28
5,MACAU,1614373203,2021-02-26,10.41,10.32,10.45,10.275,161965,2021-02-28
6,NVSAU,1614373204,2021-02-26,10.67,10.53,10.86,10.490,908956,2021-02-28


In [ ]:
# add performance to database
# connection = engine.connect()
# performance_df.to_sql('performance', con=engine, if_exists='append', index=False)

In [ ]:
# pull for one stock example
# r = requests.get(f'https://query2.finance.yahoo.com/v8/finance/chart/DFH?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1=1514903400&period2=1608357600&events=div%7Csplit&corsDomain=finance.yahoo.com')
# data = r.json()
# print(r)

# timestamp = data["chart"]["result"][0]["timestamp"]
# stk_open = data["chart"]["result"][0]["indicators"]["quote"][0]["open"]
# stk_close = data["chart"]["result"][0]["indicators"]["quote"][0]["close"]
# stk_high = data["chart"]["result"][0]["indicators"]["quote"][0]["high"]
# stk_low = data["chart"]["result"][0]["indicators"]["quote"][0]["low"]
# stk_vol = data["chart"]["result"][0]["indicators"]["quote"][0]["volume"]

# # dataframe with new stock info
# df = pd.DataFrame({"symbol" : "RSVAU",
#                    "unix_time" : timestamp,
#                    "date" : [datetime.fromtimestamp(ts).strftime('%Y-%m-%d') for ts in timestamp],
#                    "open" : stk_open, 
#                    "close" : stk_close,
#                    "high" : stk_high,
#                    "low" : stk_low,
#                    "volume" : stk_vol
#                   })

# df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
# df

# Industry Performance

In [ ]:
# get unix time range for web request
start_unixtime = 1514903400  #Jan 2, 2018

current_date = datetime.now()
print(current_date)

earliest_date_time = current_date.replace(hour=0, minute=0, second=0, microsecond=0)
print(earliest_date_time)

end_unixtime = time.mktime(earliest_date_time.timetuple())
end_unixtime = int(end_unixtime)
end_unixtime

In [ ]:
today = datetime.today().strftime("%Y-%m-%d")
print(today)

In [ ]:
# industry index's list for requests
ind_idx_lookup_list = ['%5EDJI', '%5EIXIC', '%5EGSPC']
ind_idx_symbol_list = ['DJI', 'IXIC', 'GSPC']

In [ ]:
# initial industry dataframe to interate through for requests
ind_df = pd.DataFrame({'symbol_lookup': ind_idx_lookup_list,
                      'symbol': ind_idx_symbol_list})
ind_df["default_start_unixtime"] = start_unixtime
ind_df["end_unixtime"] = end_unixtime
ind_df

In [ ]:
# update to only pull new date information
ind_perf_start_unix = pd.read_sql("SELECT symbol, max(unix_time) AS max_unix_captured  FROM industry_performance  GROUP BY symbol""", connection)
ind_df = ind_df.merge(ind_perf_start_unix, on="symbol", how="outer")
ind_df["max_unix_captured"] = ind_df["max_unix_captured"].fillna(0).astype('int64')
ind_df["max_unix_captured"] = ind_df["max_unix_captured"] + 86400  #add a day to latest date captured
ind_df

In [ ]:
ind_df["start_unixtime"] = ind_df[["default_start_unixtime", "max_unix_captured"]].max(axis=1).astype('int64') 
ind_df

In [ ]:
# empty list of dfs
performance_df_list= []

# make requests for performance information for each row
for row in ind_df.itertuples():

    url = f'https://query2.finance.yahoo.com/v8/finance/chart/{row.symbol_lookup}?formatted=true&crumb=T18HKACbWPn&lang=en-US&region=US&includeAdjustedClose=true&interval=1d&period1={row.start_unixtime}&period2={row.end_unixtime}&events=div%7Csplit&corsDomain=finance.yahoo.com'
    r = requests.get(url)
    print(f"trying url: {url}")
    if r.ok:
        try: 
            data = r.json()

            # get data
            timestamp = data["chart"]["result"][0]["timestamp"]
            stk_open = data["chart"]["result"][0]["indicators"]["quote"][0]["open"]
            stk_close = data["chart"]["result"][0]["indicators"]["quote"][0]["close"]
            stk_high = data["chart"]["result"][0]["indicators"]["quote"][0]["high"]
            stk_low = data["chart"]["result"][0]["indicators"]["quote"][0]["low"]
            stk_vol = data["chart"]["result"][0]["indicators"]["quote"][0]["volume"]

            #transform into dataframe
            df = pd.DataFrame({"symbol" : row.symbol,
                               "unix_time" : timestamp,
                               "date" : [datetime.fromtimestamp(ts).strftime('%Y-%m-%d') for ts in timestamp],
                               "open" : stk_open, 
                               "close" : stk_close,
                               "high" : stk_high,
                               "low" : stk_low,
                               "volume" : stk_vol
                              })

            df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
            df['date_pulled'] = today

            print(f"{row.symbol} has results")
            performance_df_list.append(df)
        except KeyError:
            continue
    
    else:
        print(f"{row.symbol} NO RESULTS")

    time.sleep(.5)
    

In [ ]:
ind_performance_df = pd.concat(performance_df_list)
ind_performance_df

In [ ]:
# add performance to database
connection = engine.connect()
ind_performance_df.to_sql('industry_performance', con=engine, if_exists='append', index=False)

# Company Characteristics
This will provide city, industry, etc.  
https://query1.finance.yahoo.com/v10/finance/quoteSummary/AAPL?modules=assetProfile%2CearningsHistory

In [ ]:
# get symbols from SQL
connection = engine.connect()
symbols_df = pd.read_sql("SELECT symbol FROM stocks", connection)
symbols_df.head()

In [ ]:
# get symbols that already have company info
symb_in_compinfo = pd.read_sql("SELECT symbol FROM company_info", connection)
symb_in_compinfo.head()

In [ ]:
# remove any symbols company info already avaialable for, only keeping "new" symbols
symbols_df = symbols_df[~symbols_df["symbol"].isin(symb_in_compinfo["symbol"])]
symbols_df

In [ ]:
#get characteristics dataframe
today = datetime.today().strftime("%Y-%m-%d")

# empty list for df
symbol = []
address = []
city = []
state = []
zip_code = []
country = []
website = []
industry = []
sector = []
business_summary = []

# make requests for characteristic information for each row
for row in symbols_df.itertuples():

    url = f'https://query1.finance.yahoo.com/v10/finance/quoteSummary/{row.symbol}?modules=assetProfile%2CearningsHistory'
    r = requests.get(url)
    print(f"trying url: {url}")
    if r.ok:
        try: 
            data = r.json()
            char_dict = data["quoteSummary"]["result"][0]["assetProfile"]

            symbol.append(row.symbol)
            address.append(char_dict.get("address1", ""))
            city.append(char_dict.get("city", ""))
            state.append(char_dict.get("state", ""))  
            zip_code.append(char_dict.get("zip", ""))  
            country.append(char_dict.get("country", ""))  
            website.append(char_dict.get("website", ""))
            industry.append(char_dict.get("industry", ""))  
            sector.append(char_dict.get("sector", ""))  
            
            # shorten the character length of the full business summary
            bus_summary = char_dict.get("longBusinessSummary", "")
            if len(bus_summary) > 252:
                bus_summary = bus_summary[:995] + '...'
            business_summary.append(bus_summary)
                

        except KeyError:
            continue
    
    else:
        print(f"{row.symbol} NO RESULTS")

    time.sleep(.25)

#transform into dataframe
comp_info_df = pd.DataFrame({"symbol": symbol,
                   "address": address,
                   "city": city,
                   "state": state,
                   "zip_code": zip_code,
                   "country": country,
                   "website": website,
                   "industry": industry,
                   "sector": sector,
                   "business_summary": business_summary})
comp_info_df['date_pulled'] = today
comp_info_df.head()


In [ ]:
# add company info to database
connection = engine.connect()
comp_info_df.to_sql('company_info', con=engine, if_exists='append', index=False)

In [ ]:
# get single characteristics for testing
# symbol = "VTAQU"
# url = f'https://query1.finance.yahoo.com/v10/finance/quoteSummary/{symbol}?modules=assetProfile%2CearningsHistory'
# r = requests.get(url)
# data = r.json()
# char_dict = data["quoteSummary"]["result"][0]["assetProfile"]

# address = char_dict.get("address1", "")
# city = char_dict.get("city", "")
# state = char_dict.get("state", "")  
# zip_code = char_dict.get("zip", "")  
# country = char_dict.get("country", "")  
# website = char_dict.get("website", "")
# industry = char_dict.get("industry", "")  
# sector = char_dict.get("sector", "")  
# business_summary = char_dict.get("longBusinessSummary", "")  

# characteristics_df = pd.DataFrame({"symbol": symbol,
#                                    "address": address,
#                                    "city": city,
#                                    "state": state,
#                                    "zip_code": zip_code,
#                                    "country": country,
#                                    "website": website,
#                                    "industry": industry,
#                                    "sector": sector,
#                                    "business_summary": business_summary
#                                   }
#                                   ,index=[0]
#                                   )
# characteristics_df

# Update Company Characteristics

In [ ]:
# query characteristics with NULL or blank values for industry or sector
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')
connection = engine.connect()
ci_limited_df = pd.read_sql("""
                        SELECT *
                        FROM company_info
                        WHERE industry IS NULL 
                            OR industry = ''
                            OR sector IS NULL
                            OR sector = ''""", connection)
connection.close()
ci_limited_df.head()

In [ ]:
# create engine
engine = create_engine(f'mysql://{dbuser}:{dbpass}@{dbhost}/{dbname}?charset=utf8')

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Print all of the classes mapped to the Base
Base.classes.keys()

In [ ]:
# Assign the company_info class to a variable called `Company_Info`
Comp_Info = Base.classes.company_info

# Create a session
session = Session(engine)

In [ ]:
# Use the session to query company info table and display the first 5 symbols
for r in session.query(Comp_Info.symbol, Comp_Info.address).limit(5).all():
    print(r)

In [ ]:
#get characteristics dataframe
today = datetime.today().strftime("%Y-%m-%d")

# empty list for df
symbol = []
address = []
city = []
state = []
zip_code = []
country = []
website = []
industry = []
sector = []
business_summary = []

# make requests for characteristic information for each row
for row in ci_limited_df.itertuples():

    url = f'https://query1.finance.yahoo.com/v10/finance/quoteSummary/{row.symbol}?modules=assetProfile%2CearningsHistory'
    r = requests.get(url)
    print(f"trying url: {url}")
    if r.ok:
        try: 
            data = r.json()
            char_dict = data["quoteSummary"]["result"][0]["assetProfile"]

            symbol.append(row.symbol)
            address.append(char_dict.get("address1", ""))
            city.append(char_dict.get("city", ""))
            state.append(char_dict.get("state", ""))  
            zip_code.append(char_dict.get("zip", ""))  
            country.append(char_dict.get("country", ""))  
            website.append(char_dict.get("website", ""))
            industry.append(char_dict.get("industry", ""))  
            sector.append(char_dict.get("sector", ""))  
            
            # shorten the character length of the full business summary
            bus_summary = char_dict.get("longBusinessSummary", "")
            if len(bus_summary) > 252:
                bus_summary = bus_summary[:995] + '...'
            business_summary.append(bus_summary)
                

        except KeyError:
            continue
    
    else:
        print(f"{row.symbol} NO RESULTS")

    time.sleep(.25)

#transform into dataframe
comp_info_df = pd.DataFrame({"symbol": symbol,
                   "address": address,
                   "city": city,
                   "state": state,
                   "zip_code": zip_code,
                   "country": country,
                   "website": website,
                   "industry": industry,
                   "sector": sector,
                   "business_summary": business_summary})
comp_info_df['date_pulled'] = today
comp_info_df.head()


In [ ]:
# get dataframe for company info that has industry or sector and previously didn't
df = comp_info_df.loc[(comp_info_df['industry'] != '')  | (comp_info_df['sector'] != '')]
df

In [ ]:
for row in df.itertuples():

    # Get the record we want to change
    stock = session.query(Comp_Info).filter(Comp_Info.symbol==row.symbol).first()

    # Change the record
    stock.address = df.loc[df["symbol"] == row.symbol, 'address'].iloc[0]
    stock.city = df.loc[df["symbol"] == row.symbol, 'city'].iloc[0]
    stock.state = df.loc[df["symbol"] == row.symbol, 'state'].iloc[0]
    stock.zip_code = df.loc[df["symbol"] == row.symbol, 'zip_code'].iloc[0]
    stock.country = df.loc[df["symbol"] == row.symbol, 'country'].iloc[0]
    stock.industry = df.loc[df["symbol"] == row.symbol, 'industry'].iloc[0]
    stock.sector = df.loc[df["symbol"] == row.symbol, 'sector'].iloc[0]
    stock.business_summary = df.loc[df["symbol"] == row.symbol, 'business_summary'].iloc[0]
    stock.date_pulled = df.loc[df["symbol"] == row.symbol, 'date_pulled'].iloc[0]

# Update the database
session.commit()

# Market Cap

In [ ]:
# get unix time range for web request
start_unixtime = 1514903400  #Jan 2, 2018

current_date = datetime.now()
print(current_date)

earliest_date_time = current_date.replace(hour=0, minute=0, second=0, microsecond=0)
print(earliest_date_time)

end_unixtime = time.mktime(earliest_date_time.timetuple())
end_unixtime = int(end_unixtime)
end_unixtime

In [ ]:
# query stocks from SQL
connection = engine.connect()
ipo_stocks = pd.read_sql("SELECT symbol, deal_status FROM stocks WHERE deal_status = 'priced'", connection)
ipo_stocks["default_start_unixtime"] = start_unixtime
ipo_stocks["end_unixtime"] = end_unixtime
ipo_stocks

In [ ]:
# update to unix to avoid capturing duplicate records
comp_fin_start_unix = pd.read_sql("SELECT symbol, max(unix_timestamp) AS max_unix_captured  FROM market_cap  GROUP BY symbol""", connection)
ipo_stocks = ipo_stocks.merge(comp_fin_start_unix, on="symbol", how="outer")
ipo_stocks["max_unix_captured"] = ipo_stocks["max_unix_captured"].fillna(0).astype('int64')
ipo_stocks["max_unix_captured"] = ipo_stocks["max_unix_captured"] + 604800  #add a week to latest date captured
ipo_stocks

In [ ]:
ipo_stocks["start_unixtime"] = ipo_stocks[["default_start_unixtime", "max_unix_captured"]].max(axis=1).astype('int64') 
ipo_stocks

In [ ]:
def fn_get_fin_metric_index(df, metric):
    """returns the index for a given metric is in the list of metrics on yahoo (cannot use a key as it is not a dictionary).
    For example, the metric parameteter may be 'trailingMarketCap'. Other options include:
    
    trailingMarketCap, trailingForwardPeRatio, trailingPeRatio, trailingEnterpriseValue, trailingPegRatio, 
    trailingEnterprisesValueRevenueRatio, trailingEnterprisesValueEBITDARatio, trailingPsRatio, trailingPbRatio
    """
    idx = df[df["metric"] == metric].index
    idx = idx[0]
    return idx

In [ ]:
# rolling 12 fields are trailingMarketCap, trailingForwardPeRatio, trailingPeRatio, trailingEnterpriseValue, trailingPegRatio, 
# trailingEnterprisesValueRevenueRatio, trailingEnterprisesValueEBITDARatio, trailingPsRatio, trailingPbRatio

In [ ]:
#get market cap dataframe
today = datetime.today().strftime("%Y-%m-%d")

# empty list for df
symbol_list = []
unix_timestamp_list = []
date_list = []
market_cap_list = []
market_cap_fmt_list = []

# make requests for characteristic information for each row
for row in ipo_stocks.itertuples():

    #NOTE NOTE NOTE: WILL NEED UP UPDATE START UNIXTIME AFTER INITIAL RUN
    url = f'https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/{row.symbol}?lang=en-US&region=US&symbol={row.symbol}&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2CtrailingForwardPeRatio%2CquarterlyPegRatio%2CtrailingPegRatio%2CquarterlyPsRatio%2CtrailingPsRatio%2CquarterlyPbRatio%2CtrailingPbRatio%2CquarterlyEnterprisesValueRevenueRatio%2CtrailingEnterprisesValueRevenueRatio%2CquarterlyEnterprisesValueEBITDARatio%2CtrailingEnterprisesValueEBITDARatio&merge=false&period1={row.start_unixtime}&period2={row.end_unixtime}&corsDomain=finance.yahoo.com'
    r = requests.get(url)
    
    if r.ok:
        time.sleep(.25)
        data = r.json()
        
        payload = data["timeseries"]["result"]
        index_list = []
        metric_list = []
        count = 0

        for d in range(0, len(payload)):
            metric = payload[d]["meta"]["type"][0] 
            metric_list.append(metric) 
            index_list.append(count)
            count += 1
            
#         print(f"{index_list} : {metric_list}")
        comp_fin_df = pd.DataFrame({"metric": metric_list},
                          index=index_list)
        

        print(f"trying url: {url}")
        try: 
            for n in range(0, len(payload[fn_get_fin_metric_index(comp_fin_df, "trailingMarketCap")]["trailingMarketCap"])):
                symbol_list.append(row.symbol)


                # get the market cap index using the fn_get_fin_metric_index function
                market_cap = payload[fn_get_fin_metric_index(comp_fin_df, "trailingMarketCap")]["trailingMarketCap"]

                unix_timestamp_list.append(payload[fn_get_fin_metric_index(comp_fin_df, "trailingMarketCap")]["timestamp"][n])
                date_list.append(market_cap[n]["asOfDate"])
                market_cap_list.append(market_cap[n]["reportedValue"]["raw"])
                market_cap_fmt_list.append(market_cap[n]["reportedValue"]["fmt"])
        except KeyError:
            continue

In [ ]:
#transform into dataframe
market_cap_df = pd.DataFrame({"symbol": symbol_list,
                                   "unix_timestamp": unix_timestamp_list,
                                   "date": date_list,
                                   "market_cap": market_cap_list,
                                   "market_cap_formatted": market_cap_fmt_list, 
                                   "date_pulled": today
                                  })

market_cap_df["market_cap"] = market_cap_df["market_cap"].astype('int64')
market_cap_df

In [ ]:
# add company info to database
connection = engine.connect()
market_cap_df.to_sql('market_cap', con=engine, if_exists='append', index=False)

In [ ]:
# get single characteristics for testing
# symbol = "AAPL"

# symbol_list = []
# unix_timestamp_list = []
# date_list = []
# market_cap_list = []
# market_cap_fmt_list = []


# url = f'https://query1.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/{symbol}?lang=en-US&region=US&symbol=AAPL&padTimeSeries=true&type=quarterlyMarketCap%2CtrailingMarketCap%2CquarterlyEnterpriseValue%2CtrailingEnterpriseValue%2CquarterlyPeRatio%2CtrailingPeRatio%2CquarterlyForwardPeRatio%2CtrailingForwardPeRatio%2CquarterlyPegRatio%2CtrailingPegRatio%2CquarterlyPsRatio%2CtrailingPsRatio%2CquarterlyPbRatio%2CtrailingPbRatio%2CquarterlyEnterprisesValueRevenueRatio%2CtrailingEnterprisesValueRevenueRatio%2CquarterlyEnterprisesValueEBITDARatio%2CtrailingEnterprisesValueEBITDARatio&merge=false&period1=1514903400&period2=1608962400&corsDomain=finance.yahoo.com'
# r = requests.get(url)
# data = r.json()

# for n in range(0, len(payload[fn_get_fin_metric_index(comp_fin_df, "trailingMarketCap")]["trailingMarketCap"])):
#     symbol_list.append(symbol)
    
    
#     # get the market cap index using the fn_get_fin_metric_index function
#     market_cap = payload[fn_get_fin_metric_index(comp_fin_df, "trailingMarketCap")]["trailingMarketCap"]
    
#     unix_timestamp_list.append(payload[fn_get_fin_metric_index(comp_fin_df, "trailingMarketCap")]["timestamp"][n])
#     date_list.append(market_cap[n]["asOfDate"])
#     period_type_list.append(market_cap[n]["periodType"])
#     market_cap_list.append(market_cap[n]["reportedValue"]["raw"])
#     market_cap_fmt_list.append(market_cap[n]["reportedValue"]["fmt"])
    
# comp_market_cap_df = pd.DataFrame({"symbol": symbol_list,
#                                    "unix_timestamp": unix_timestamp_list,
#                                    "date": date_list,
#                                    "market_cap": market_cap_list,
#                                    "market_cap_formatted": market_cap_fmt_list, 
#                                    "date_pulled": today
#                                   })

# comp_market_cap_df["market_cap"] = comp_market_cap_df["market_cap"].astype('int64')
# comp_market_cap_df

# Analysis

Information that may be interesting to share.  Examples include:
1. Timing of when it launches, how long its been, etc.
2. Price performance
    - Launch date open and close price (how they did on first day)
    - How did it do when its hit 1 month, 3 month, 6 month, 1 year milestone
3. Industry perormance
    - Did it outperform the S&P 
    - Did it outperform they sector (Ex: tech, consumer goods)
4. Top performers
    - Which IPOs did best in last 1 month, 3 month, 6 month, 1 year milestone

In [ ]:
# Print text for each record

for i in range(0,len(response)):
  print(f"{response[i]['expected_to_trade']}: {response[i]['company']} [{response[i]['symbol_proposed']}]. Price (Low-High): ${response[i]['price_low']}-{response[i]['price_high']}. #new_ipo_{response[i]['symbol_proposed']}")  

In [ ]:
# Establish Twitter connection

import tweepy

CONSUMER_KEY = "consumer_key"
CONSUMER_SECRET = "consumer_secret"   
ACCESS_KEY = "access_key"    
ACCESS_SECRET = "access_secret"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

api = tweepy.API(auth)

In [ ]:
# Post a tweet for each record
for i in range(0,len(response)):
  new_tweet = f"{response[i]['expected_to_trade']}: {response[i]['company']} [{response[i]['symbol_proposed']}]. Price (Low-High): ${response[i]['price_low']}-{response[i]['price_high']}. #new_ipo_{response[i]['symbol_proposed']}" 
  api.update_status(new_tweet)      